In [1]:
import os
import json
import random
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import re
from google.colab import drive
import matplotlib.pyplot as plt

In [2]:
piece_mappings = {
    "P": 1,
    "N": 2,
    "B": 3,
    "R": 4,
    "Q": 5,
    "K": 6,
    "p": -1,
    "n": -2,
    "b": -3,
    "r": -4,
    "q": -5,
    "k": -6,
}
board_mappings = {
    "a": 0,
    "b": 8,
    "c": 16,
    "d": 24,
    "e": 32,
    "f": 40,
    "g": 48,
    "h": 56
}
BATCH_SIZE = 256
BUFFER_SIZE = 20000

In [29]:

def plot_values(epochs, acc):

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.title('Training accuracy')
    plt.legend()

    plt.figure()

    plt.savefig('/content/drive/MyDrive/accuracy.png')

    plt.show()

def create_model(num_hneurons, model_optimizer):
    model = keras.Sequential()
    num_hlayers = len(num_hneurons)
    input_shape = (70,)

    model.add(layers.InputLayer(input_shape=input_shape))

    for i in range(1, num_hlayers):
        model.add(layers.Dense(num_hneurons[i], activation="relu"))

    model.add(layers.Dense(1, activation="linear"))

    model.compile(optimizer=model_optimizer, loss="mean_squared_error", metrics=["accuracy"])

    return model

@tf.function
def train_step(data):
    x, y = data

    with tf.GradientTape() as tape:
        y_pred = model(x, training=True)  # Forward pass
        loss = model.compute_loss(y=y, y_pred=y_pred)

    trainable_vars = model.trainable_variables
    gradients = tape.gradient(loss, trainable_vars)

    model.optimizer.apply_gradients(zip(gradients, trainable_vars))

    for metric in model.metrics:
        if metric.name == "loss":
            metric.update_state(loss)
        else:
            metric.update_state(y, y_pred)

    return {m.name: m.result() for m in model.metrics}

def train(dataset, epochs):
    for epoch in range(epochs):
        print(f"start Epoch {epoch}")
        start = time.time()

        for data_batch in dataset:
            train_step(data_batch)

        if (epoch+1)%10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

        print(model.evaluate(test_inp, test_outp))
        print(f"Epoch {epoch} finished in {time.time()-start}")

def fit_model(model, dataset):
    epochs = 200

    with tf.device('/device:GPU:0'):
        print(0)
        history = model.fit(dataset, epochs=epochs)
        # history = model.fit(dataset)
        # train(dataset, epochs)

    # result = model.evaluate(input_data[0:100], output_data[0:100])

    # print(f"result: {result}")
    print("Done")
    plot_values(epochs, history.history["accuracy"])

def process_data(json_data):
    clean_input = []
    clean_output = []

    for position in json_data["data"]:
        fen = position["fen"].split()
        player = 1 if fen[1] == "w" else -1  # 1 for white, -1 for black

        best_evaluation = -player*float("inf")  # initially worst evaluation for current player

        for line in position["evals"][0]["pvs"]:
            # if color*best_evaluation == float("inf"):
            #     continue

            cp = line.get("cp")
            mate = line.get("mate")

            if cp is None:
                if mate is not None and player*mate > 0:
                    best_evaluation = player*float("inf")
            elif player == 1 and cp > best_evaluation or player == -1 and best_evaluation > cp:
                best_evaluation = cp

        matrix_fen = []


        for square in fen[0].replace("/", ""):
            if square in piece_mappings.keys():
                matrix_fen.append(piece_mappings.get(square))
            else:
                matrix_fen += [0]*int(square)

        matrix_fen += [
            player,
            int("K" in fen[2]),
            int("Q" in fen[2]),
            int("k" in fen[2]),
            int("q" in fen[2]),
            -1 if fen[3] == "-" else board_mappings[fen[3][0]] + int(fen[3][1])-1,
        ]

        clean_input.append(matrix_fen)

        if best_evaluation == +float("inf"):
            clean_output.append(1000000)
        elif best_evaluation == -float("inf"):
            clean_output.append(-1000000)
        else:
            clean_output.append(best_evaluation)

    return clean_input, clean_output

In [7]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [4]:
drive.mount("/content/drive")

drive_filepath = "/content/drive/MyDrive/usable_lichess_db_eval1.json"
with open(drive_filepath) as json_file:
    json_data = json.load(json_file)

input_data, output_data = process_data(json_data)
dataset = tf.data.Dataset.from_tensor_slices((input_data, output_data)).batch(BATCH_SIZE)

del json_data
del input_data
del output_data

print(dataset)

Mounted at /content/drive
<_BatchDataset element_spec=(TensorSpec(shape=(None, 70), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [22]:
# optimizer = keras.optimizers.Adam(1e-4)
# model = create_model([70, 128, 32], optimizer)

checkpoint_dir = "/content/drive/MyDrive/checkpoints2"
checkpoint_prefix = f"{checkpoint_dir}/ckpt"
checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)

test_inp, test_outp = next(iter(dataset))

In [9]:
train(dataset, 100)

start Epoch 0
Epoch 0 finished in 44.8333101272583
start Epoch 1
Epoch 1 finished in 41.7439603805542
start Epoch 2
Epoch 2 finished in 41.789498805999756
start Epoch 3
Epoch 3 finished in 41.546268701553345
start Epoch 4
Epoch 4 finished in 41.95014762878418
start Epoch 5
Epoch 5 finished in 41.56672692298889
start Epoch 6
Epoch 6 finished in 41.608290672302246
start Epoch 7
Epoch 7 finished in 41.375091791152954
start Epoch 8
Epoch 8 finished in 41.50683784484863
start Epoch 9
Epoch 9 finished in 41.83117604255676
start Epoch 10
Epoch 10 finished in 41.654293060302734
start Epoch 11
Epoch 11 finished in 41.566831827163696
start Epoch 12
Epoch 12 finished in 41.56121850013733
start Epoch 13
Epoch 13 finished in 41.40241861343384
start Epoch 14
Epoch 14 finished in 41.49090528488159
start Epoch 15
Epoch 15 finished in 41.677581787109375
start Epoch 16
Epoch 16 finished in 41.53267788887024
start Epoch 17
Epoch 17 finished in 41.588470697402954
start Epoch 18
Epoch 18 finished in 41.241

In [31]:
model.save("/content/drive/MyDrive/model_300.h5")
model.save("/content/drive/MyDrive/model_300.keras")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
test_inp, test_outp = next(iter(dataset))
print(test_inp, test_outp)
print(model.evaluate(test_inp, test_outp))
print(dataset)

tf.Tensor(
[[-4 -2 -3 ...  1  1 -1]
 [-4 -2 -3 ...  1  1 -1]
 [ 0  0  4 ...  0  0 -1]
 ...
 [-4 -2  0 ...  1  1 -1]
 [-4 -2 -3 ...  1  1 -1]
 [-4 -2 -3 ...  1  1 -1]], shape=(256, 70), dtype=int32) tf.Tensor(
[      20       64        0        0      994       20       31        0
       13        0       60     -131     -767      -64       58     -815
      -94      498       35       39       31       53       59       12
      778      483       11       39     -116       41      148       21
       33       -6        0     -740       87        0       69       31
  1000000       79      789       42       39        0       64       10
       75      789       72      -21    -6180      728 -1000000      627
       -8     -701       58      144       36       32       96      516
      994     -723      724       39       14      379       34      -21
     -700       49      616      -86      -37        0      201      -11
       98      728     1109      137       75       73       

In [30]:
fit_model(model, dataset)

0
Epoch 1/200
22673/22673 [==============================] - 61s 3ms/step - loss: 55939047424.0000 - accuracy: 0.0327
Epoch 2/200
22673/22673 [==============================] - 59s 3ms/step - loss: 56172412928.0000 - accuracy: 0.0323
Epoch 3/200
22673/22673 [==============================] - 59s 3ms/step - loss: 56124227584.0000 - accuracy: 0.0325
Epoch 4/200
22673/22673 [==============================] - 59s 3ms/step - loss: 56057860096.0000 - accuracy: 0.0327
Epoch 5/200
22673/22673 [==============================] - 59s 3ms/step - loss: 55981101056.0000 - accuracy: 0.0328
Epoch 6/200
22673/22673 [==============================] - 59s 3ms/step - loss: 55894097920.0000 - accuracy: 0.0330
Epoch 7/200
22673/22673 [==============================] - 59s 3ms/step - loss: 55796285440.0000 - accuracy: 0.0332
Epoch 8/200
22673/22673 [==============================] - 59s 3ms/step - loss: 55691833344.0000 - accuracy: 0.0334
Epoch 9/200
22673/22673 [==============================] - 59s 3ms/ste

KeyError: 'acc'